In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook
import tables
import math

In [2]:
result_predictions="ensemble.h5"

In [3]:
with tables.open_file(result_predictions, mode='r') as hdf5_file:
    print(hdf5_file)

ensemble.h5 (File) ''
Last modif.: 'Thu Dec 14 10:52:56 2017'
Object Tree: 
/ (RootGroup) ''
/keras_incepresv2_max (EArray(1768182, 5270)) ''
/keras_incepresv2_mean (EArray(1768182, 5270)) ''
/keras_incepresv2_median (EArray(1768182, 5270)) ''
/keras_incepv2_max (EArray(1768182, 5270)) ''
/keras_incepv2_mean (EArray(1768182, 5270)) ''
/keras_incepv2_median (EArray(1768182, 5270)) ''
/keras_resnet101_max (EArray(1768182, 5270)) ''
/keras_resnet101_mean (EArray(1768182, 5270)) ''
/keras_resnet101_median (EArray(1768182, 5270)) ''
/keras_xcp_max (EArray(1768182, 5270)) ''
/keras_xcp_mean (EArray(1768182, 5270)) ''
/keras_xcp_median (EArray(1768182, 5270)) ''
/pytorch_incepv3_max (EArray(1768182, 5270)) ''
/pytorch_incepv3_mean (EArray(1768182, 5270)) ''
/pytorch_incepv3_median (EArray(1768182, 5270)) ''
/pytorch_resnet101_max (EArray(1768182, 5270)) ''
/pytorch_resnet101_mean (EArray(1768182, 5270)) ''
/pytorch_resnet101_median (EArray(1768182, 5270)) ''
/pytorch_resnext_max (EArray(17681

In [4]:
def return_idx(start, stop, step):
    x=list(range(start, stop, step))
    y=list(range(start+step, stop, step))+[stop]
    for i in range(len(y)):
        yield x[i], y[i]

In [5]:
final=[]

with tables.open_file(result_predictions, mode='r') as hdf5_file:
    start=0
    end=len(hdf5_file.root.pytorch_xcp_median)
    chunksize=800   
    
    indices=return_idx(start, end, chunksize)
    total_batches = math.ceil(end/chunksize)
    
    bar = tqdm_notebook(total=total_batches)
    
    for start, end in indices:     
        final[start:end] = np.argmax(hdf5_file.root.keras_incepv2_mean[start:end]+
                                      hdf5_file.root.keras_resnet101_mean[start:end]+
                                      hdf5_file.root.keras_incepresv2_mean[start:end]+
                                      hdf5_file.root.keras_xcp_mean[start:end]+
                                      hdf5_file.root.pytorch_se_incepv3_mean[start:end]+
                                      hdf5_file.root.pytorch_incepv3_mean[start:end]+
                                      hdf5_file.root.pytorch_xcp_mean[start:end]+
                                      hdf5_file.root.pytorch_resnext_mean[start:end]+
                                      hdf5_file.root.pytorch_resnet101_mean[start:end], 
                                      axis=1)       
        bar.update()

A Jupyter Widget

In [6]:
result=pd.DataFrame()
result["_id"]=pd.read_csv("argmax.csv")["_id"].values
result["final"]=final

In [7]:
result.to_csv("argmax.csv", index=False)

In [8]:
result.shape

(1768182, 2)

In [9]:
result.head()

,_id,final
0,10,601
1,14,5055
2,21,5055
3,24,3844
4,27,1465
